# brian2-symbode — Interactive Walkthrough

This notebook demonstrates the `SymbolicGraphExtractor` step by step.

**What you'll learn:**
1. How Brian2 stores equations across NeuronGroups and Synapses
2. How `resolve_variable` recursively walks the object graph
3. How summed variables, subexpressions, and synaptic references are handled
4. How `get_derived_ode` produces a flat symbolic expression
5. How `get_structured_ode` separates local dynamics from coupling terms
6. How dead summed variables are detected
7. How `discretise` converts continuous ODEs to discrete update maps
8. How the full Jacobian is computed for stability analysis

## 1. Setup — Define a Rate-Model Equation

We use a rate-model neuron with 4 summed synaptic currents and a custom transfer function:

```
dx/dt = (-x + I_EE + I_EI + I_IE + I_II) / tau
I_EE, I_EI, I_IE, I_II : 1   <- summed variables (written by synapses)
tau : second (shared)
g : 1                          <- per-neuron gain
r_0, r_max : shared, constant  <- transfer function bounds
r = f(x, g, r_0, r_max) : 1   <- subexpression (firing rate)
```

Both Exc and Inh groups use the **same equation**, but only some synapses
target each group — creating "dead" summed variables.

In [1]:
import numpy as np
import sympy as sp
from brian2 import *

from brian2_symbode import SymbolicGraphExtractor, get_derived_ode, compute_full_jacobian

# --- Equation string ---
eqs = '''
dx/dt = (-x + I_EE + I_EI + I_IE + I_II) / tau : 1
I_EE : 1
I_EI : 1
I_IE : 1
I_II : 1
tau : second (shared)
g : 1
r_0 : 1 (shared, constant)
r_max : 1 (shared, constant)
r = f(x, g, r_0, r_max) : 1
'''

# --- Symbolic transfer function (SymPy replacement for Brian2's C++/CUDA 'f') ---
def f_symbolic(x, g, r_0, r_max):
    span = r_max - r_0
    return sp.Piecewise(
        (r_0, x < 0),
        (r_0 + span * sp.tanh((g * x) / span), True),
    )

# Show what f_symbolic produces
x_test, g_test, r0_test, rmax_test = sp.symbols('x g r_0 r_max')
print("f_symbolic(x, g, r_0, r_max) =")
sp.pprint(f_symbolic(x_test, g_test, r0_test, rmax_test))

f_symbolic(x, g, r_0, r_max) =
⎧                r₀                  for x < 0
⎪                                             
⎨                      ⎛   g⋅x    ⎞           
⎪r₀ + (-r₀ + rₘₐₓ)⋅tanh⎜──────────⎟  otherwise
⎩                      ⎝-r₀ + rₘₐₓ⎠           


## 2. Build a Minimal Network

6 Exc + 4 Inh neurons, fully connected within and between populations.

In [2]:
start_scope()

M = 6   # Excitatory
N = 10  # Total (6 Exc + 4 Inh)

# --- Neuron groups ---
exc = NeuronGroup(M, eqs, threshold='True', reset='',
                  name='soc_exc_neurons', method='euler')
inh = NeuronGroup(N - M, eqs, threshold='True', reset='',
                  name='soc_inh_neurons', method='euler')

for grp in [exc, inh]:
    grp.tau = 200 * ms
    grp.g = 1.0
    grp.r_0 = 20.0
    grp.r_max = 100.0

# --- Synapses (XY = Source X -> Target Y) ---
EE_Syn = Synapses(exc, exc, model='we : 1\nI_EE_post = we*r_pre : 1 (summed)',
                  name='EE_Synapses', method='euler')
EI_Syn = Synapses(exc, inh, model='we : 1\nI_EI_post = we*r_pre : 1 (summed)',
                  name='EI_Synapses', method='euler')
IE_Syn = Synapses(inh, exc, model='wi : 1\nI_IE_post = wi*r_pre : 1 (summed)',
                  name='IE_Synapses', method='euler')
II_Syn = Synapses(inh, inh, model='wi : 1\nI_II_post = wi*r_pre : 1 (summed)',
                  name='II_Synapses', method='euler')

np.random.seed(42)
for syn in [EE_Syn, EI_Syn, IE_Syn, II_Syn]:
    syn.connect()
    if 'we' in syn.equations.names:
        syn.we = np.random.uniform(0.1, 0.5, len(syn))
    else:
        syn.wi = np.random.uniform(-0.5, -0.1, len(syn))

net = Network(exc, inh, EE_Syn, EI_Syn, IE_Syn, II_Syn)

print(f"Network objects: {[obj.name for obj in net.objects]}")
print(f"Exc: {exc.name} ({len(exc)} neurons), Inh: {inh.name} ({len(inh)} neurons)")
print(f"Synapses: EE={len(EE_Syn)}, EI={len(EI_Syn)}, IE={len(IE_Syn)}, II={len(II_Syn)}")

Network objects: ['IE_Synapses', 'EI_Synapses', 'EE_Synapses', 'soc_exc_neurons', 'II_Synapses', 'soc_inh_neurons']
Exc: soc_exc_neurons (6 neurons), Inh: soc_inh_neurons (4 neurons)
Synapses: EE=36, EI=24, IE=24, II=16


## 3. Inspect Brian2's Internal Representations

Key data structures the extractor reads from:
- **`group.equations`**: dict of variable name -> `SingleEquation` with `.type`, `.expr`, `.flags`
- **`synapse.summed_updaters`**: dict mapping summed variable names -> `SummedVariableUpdater` with `.expression`
- **Types**: `'differential equation'`, `'subexpression'`, `'parameter'`
- **Flags**: `'constant'`, `'shared'`

In [3]:
# 3a. Equations on the Excitatory NeuronGroup
print("=" * 70)
print(f"Equations in '{exc.name}':")
print("=" * 70)
for name, eq in exc.equations.items():
    flags_str = f"  flags={set(eq.flags)}" if hasattr(eq, 'flags') and eq.flags else ""
    expr_str = f"  expr={eq.expr}" if hasattr(eq, 'expr') and eq.expr else ""
    print(f"  {name:10s} | type={eq.type:25s}{flags_str}{expr_str}")

print("\nKey: I_EE..I_II are 'parameter' type — their values come from synapses.")

Equations in 'soc_exc_neurons':
  r          | type=subexpression              expr=f(x, g, r_0, r_max)
  x          | type=differential equation      expr=(-x + I_EE + I_EI + I_IE + I_II) / tau
  I_EE       | type=parameter                
  I_EI       | type=parameter                
  I_IE       | type=parameter                
  I_II       | type=parameter                
  g          | type=parameter                
  r_0        | type=parameter                  flags={'shared', 'constant'}
  r_max      | type=parameter                  flags={'shared', 'constant'}
  tau        | type=parameter                  flags={'shared'}

Key: I_EE..I_II are 'parameter' type — their values come from synapses.


In [4]:
# 3b. Synapse summed_updaters — the hidden link
print("=" * 70)
print("Synapse summed_updaters:")
print("=" * 70)
for syn in [EE_Syn, EI_Syn, IE_Syn, II_Syn]:
    print(f"\n  '{syn.name}' ({syn.source.name} -> {syn.target.name}):")
    if hasattr(syn, 'summed_updaters'):
        for su_name, su in syn.summed_updaters.items():
            print(f"    summed['{su_name}'] = '{su.expression}'")

print("\nFor EXCITATORY group:")
print("  I_EE <- EE_Synapses  (coupling)")
print("  I_IE <- IE_Synapses  (coupling)")
print("  I_EI <- nothing      (DEAD)")
print("  I_II <- nothing      (DEAD)")

Synapse summed_updaters:

  'EE_Synapses' (soc_exc_neurons -> soc_exc_neurons):
    summed['I_EE_post'] = 'we*r_pre'

  'EI_Synapses' (soc_exc_neurons -> soc_inh_neurons):
    summed['I_EI_post'] = 'we*r_pre'

  'IE_Synapses' (soc_inh_neurons -> soc_exc_neurons):
    summed['I_IE_post'] = 'wi*r_pre'

  'II_Synapses' (soc_inh_neurons -> soc_inh_neurons):
    summed['I_II_post'] = 'wi*r_pre'

For EXCITATORY group:
  I_EE <- EE_Synapses  (coupling)
  I_IE <- IE_Synapses  (coupling)
  I_EI <- nothing      (DEAD)
  I_II <- nothing      (DEAD)


## 4. Instantiate the Extractor

The `function_map` is critical: Brian2 uses opaque C++/CUDA implementations of `f`.
We provide `f_symbolic` so the extractor can expand `f(...)` into `Piecewise(...)`.

In [5]:
extractor = SymbolicGraphExtractor(
    net,
    function_map={'f': f_symbolic},
    verbose=True
)

print(f"Objects:       {[o.name for o in extractor.objects]}")
print(f"Function map:  {list(extractor.function_map.keys())}")

Objects:       ['IE_Synapses', 'EI_Synapses', 'EE_Synapses', 'soc_exc_neurons', 'II_Synapses', 'soc_inh_neurons']
Function map:  ['f']


## 5. `resolve_variable()` — The Recursive Engine

Priority order:
1. **Summed updaters** — trace into synapse expression
2. **Equations** — diff eq -> leaf symbol; subexpression -> recurse RHS; param -> leaf
3. **Pre/post references** — `var_pre` jumps to source group
4. **Fallback** -> opaque symbol

In [6]:
# 5a. Resolve I_EE on exc — should trace into EE_Synapses -> we * r_pre -> f(...)
print("=" * 70)
print("Resolving 'I_EE' on EXCITATORY group")
print("=" * 70)
result_I_EE = extractor.resolve_variable('I_EE', exc)
print(f"\nResult: {result_I_EE}")
print(f"Free symbols: {result_I_EE.free_symbols}")

Resolving 'I_EE' on EXCITATORY group
[brian2-symbode] Resolving 'I_EE' in object 'soc_exc_neurons'...
[brian2-symbode]   -> Found 1 incoming synapse(s) writing to 'I_EE'
[brian2-symbode]     -> Tracing synapse 'EE_Synapses' with expression: we*r_pre
[brian2-symbode]       Resolving 'r_pre' in object 'EE_Synapses'...
[brian2-symbode]         -> Jumping to PRE-synaptic group 'soc_exc_neurons' for 'r'
[brian2-symbode]         Resolving 'r' in object 'soc_exc_neurons'...
[brian2-symbode]           -> Is Subexpression: f(x, g, r_0, r_max)
[brian2-symbode]           Resolving 'x' in object 'soc_exc_neurons'...
[brian2-symbode]             -> Is Differential Equation state variable. Stop recursion.
[brian2-symbode]           Resolving 'g' in object 'soc_exc_neurons'...
[brian2-symbode]             -> Is Parameter without expression.
[brian2-symbode]           Resolving 'r_0' in object 'soc_exc_neurons'...
[brian2-symbode]             -> Is Constant Parameter.
[brian2-symbode]           Resolv

In [7]:
# 5b. Resolve I_EI on exc — DEAD variable (EI_Synapses targets inh, not exc)
print("=" * 70)
print("Resolving 'I_EI' on EXCITATORY group (should be dead)")
print("=" * 70)
result_I_EI_exc = extractor.resolve_variable('I_EI', exc)
print(f"\nResult: {result_I_EI_exc}")
print("This is just a plain symbol — get_structured_ode() detects it as dead.")

Resolving 'I_EI' on EXCITATORY group (should be dead)
[brian2-symbode] Resolving 'I_EI' in object 'soc_exc_neurons'...
[brian2-symbode]   -> Is Parameter without expression.

Result: I_EI_soc_exc_neurons
This is just a plain symbol — get_structured_ode() detects it as dead.


In [8]:
# 5c. Resolve 'r' (subexpression) — should expand f(x, g, r_0, r_max)
print("=" * 70)
print("Resolving 'r' on EXCITATORY group")
print("=" * 70)
result_r = extractor.resolve_variable('r', exc)
print(f"\nResult:")
sp.pprint(result_r)
print(f"\nIs Piecewise: {result_r.has(sp.Piecewise)}")

Resolving 'r' on EXCITATORY group
[brian2-symbode] Resolving 'r' in object 'soc_exc_neurons'...
[brian2-symbode]   -> Is Subexpression: f(x, g, r_0, r_max)
[brian2-symbode]   Resolving 'x' in object 'soc_exc_neurons'...
[brian2-symbode]     -> Is Differential Equation state variable. Stop recursion.
[brian2-symbode]   Resolving 'g' in object 'soc_exc_neurons'...
[brian2-symbode]     -> Is Parameter without expression.
[brian2-symbode]   Resolving 'r_0' in object 'soc_exc_neurons'...
[brian2-symbode]     -> Is Constant Parameter.
[brian2-symbode]   Resolving 'r_max' in object 'soc_exc_neurons'...
[brian2-symbode]     -> Is Constant Parameter.

Result:
⎧                                                  r_0_soc_exc_neurons         ↪
⎪                                                                              ↪
⎨                                                                         ⎛    ↪
⎪r_0_soc_exc_neurons + (-r_0_soc_exc_neurons + r_max_soc_exc_neurons)⋅tanh⎜─── ↪
⎩                

## 6. Internal Helpers

- `_find_targeting_synapses(group, var)`: which synapses write this summed var to this group?
- `_is_summed_target_anywhere(var)`: is this var a summed target in ANY synapse?

In [9]:
print("=" * 70)
print("_find_targeting_synapses() for each I_* on BOTH groups")
print("=" * 70)

for grp in [exc, inh]:
    print(f"\n  Target: {grp.name}")
    for var in ['I_EE', 'I_EI', 'I_IE', 'I_II']:
        hits = extractor._find_targeting_synapses(grp, var)
        if hits:
            for syn, expr_str in hits:
                print(f"    {var:5s} -> found in '{syn.name}' expr='{expr_str}'")
        else:
            print(f"    {var:5s} -> not targeted (dead for this group)")

print("\n_is_summed_target_anywhere():")
for var in ['I_EE', 'I_EI', 'I_IE', 'I_II']:
    print(f"  {var}: {extractor._is_summed_target_anywhere(var)}")

_find_targeting_synapses() for each I_* on BOTH groups

  Target: soc_exc_neurons
    I_EE  -> found in 'EE_Synapses' expr='we*r_pre'
    I_EI  -> not targeted (dead for this group)
    I_IE  -> found in 'IE_Synapses' expr='wi*r_pre'
    I_II  -> not targeted (dead for this group)

  Target: soc_inh_neurons
    I_EE  -> not targeted (dead for this group)
    I_EI  -> found in 'EI_Synapses' expr='we*r_pre'
    I_IE  -> not targeted (dead for this group)
    I_II  -> found in 'II_Synapses' expr='wi*r_pre'

_is_summed_target_anywhere():
  I_EE: True
  I_EI: True
  I_IE: True
  I_II: True


## 7. `get_derived_ode()` — Flat Expression

Returns a single monolithic SymPy expression with everything inlined.
Good for Jacobians, but dead variables remain as opaque symbols.

In [10]:
ode_flat_exc = get_derived_ode(extractor, exc, 'x')
print("dx/dt for excitatory group =")
sp.pprint(ode_flat_exc)
print(f"\nFree symbols: {ode_flat_exc.free_symbols}")
print(f"Has Piecewise: {ode_flat_exc.has(sp.Piecewise)}")
print("\nNote: I_EI and I_II appear as plain symbols (dead, unresolved in flat mode).")


[brian2-symbode] Starting extraction for soc_exc_neurons [x]
[brian2-symbode] Resolving 'x' in object 'soc_exc_neurons'...
[brian2-symbode]   -> Is Differential Equation state variable. Stop recursion.
[brian2-symbode] Resolving 'tau' in object 'soc_exc_neurons'...
[brian2-symbode]   -> Is Parameter without expression.
[brian2-symbode] Resolving 'I_IE' in object 'soc_exc_neurons'...
[brian2-symbode]   -> Found 1 incoming synapse(s) writing to 'I_IE'
[brian2-symbode]     -> Tracing synapse 'IE_Synapses' with expression: wi*r_pre
[brian2-symbode]       Resolving 'wi' in object 'IE_Synapses'...
[brian2-symbode]         -> Is Parameter without expression.
[brian2-symbode]       Resolving 'r_pre' in object 'IE_Synapses'...
[brian2-symbode]         -> Jumping to PRE-synaptic group 'soc_inh_neurons' for 'r'
[brian2-symbode]         Resolving 'r' in object 'soc_inh_neurons'...
[brian2-symbode]           -> Is Subexpression: f(x, g, r_0, r_max)
[brian2-symbode]           Resolving 'x' in objec

## 8. `get_structured_ode()` — Structured Decomposition

Separates local (element-wise) dynamics from coupling (matrix-vector) terms.
Dead variables are correctly identified and zeroed out.

| Key | Description |
|-----|-------------|
| `local_expr` | Element-wise dynamics (coupling replaced with 0) |
| `coupling_terms` | One per incoming synapse: weight, source, activation |
| `dead_symbols` | Summed vars that are structurally zero for this group |
| `param_metadata` | Brian2 source, flags, sizing for each parameter |

In [11]:
structured_exc = extractor.get_structured_ode(exc, 'x')

print("--- local_expr (element-wise) ---")
sp.pprint(structured_exc['local_expr'])

print(f"\n--- state_symbol: {structured_exc['state_symbol']}")

print(f"\n--- dead_symbols (zeroed out) ---")
for ds in structured_exc['dead_symbols']:
    print(f"  {ds}")

print(f"\n--- coupling_terms ({len(structured_exc['coupling_terms'])}) ---")
for i, ct in enumerate(structured_exc['coupling_terms']):
    print(f"\n  Term {i}: {ct['synapse'].name}")
    print(f"    weight:     {ct['weight_symbol']}")
    print(f"    source:     {ct['source_group'].name} ({len(ct['source_group'])} neurons)")
    print(f"    activation:")
    sp.pprint(ct['activation_expr'], num_columns=100)

print(f"\n--- free_params ---")
for fp in structured_exc['free_params']:
    print(f"  {fp}")

[brian2-symbode] Resolving 'x' in object 'soc_exc_neurons'...
[brian2-symbode]   -> Is Differential Equation state variable. Stop recursion.
[brian2-symbode] Resolving 'tau' in object 'soc_exc_neurons'...
[brian2-symbode]   -> Is Parameter without expression.
[brian2-symbode] Resolving 'wi' in object 'IE_Synapses'...
[brian2-symbode]   -> Is Parameter without expression.
[brian2-symbode] Resolving 'r_pre' in object 'IE_Synapses'...
[brian2-symbode]   -> Jumping to PRE-synaptic group 'soc_inh_neurons' for 'r'
[brian2-symbode]   Resolving 'r' in object 'soc_inh_neurons'...
[brian2-symbode]     -> Is Subexpression: f(x, g, r_0, r_max)
[brian2-symbode]     Resolving 'x' in object 'soc_inh_neurons'...
[brian2-symbode]       -> Is Differential Equation state variable. Stop recursion.
[brian2-symbode]     Resolving 'g' in object 'soc_inh_neurons'...
[brian2-symbode]       -> Is Parameter without expression.
[brian2-symbode]     Resolving 'r_0' in object 'soc_inh_neurons'...
[brian2-symbode]  

In [12]:
# Parameter metadata — where each parameter lives in Brian2
print("=" * 70)
print("param_metadata")
print("=" * 70)
for sym_name, meta in structured_exc['param_metadata'].items():
    src_name = meta['source_obj'].name if meta['source_obj'] else 'None'
    print(f"\n  {sym_name}")
    print(f"    source: {src_name}, var: '{meta['var_name']}', type: {meta['eq_type']}")
    print(f"    flags: {meta['flags']}, per_neuron: {meta['is_per_neuron']}, size: {meta['group_size']}")

param_metadata

  g_soc_exc_neurons
    source: soc_exc_neurons, var: 'g', type: parameter
    flags: set(), per_neuron: True, size: 6

  g_soc_inh_neurons
    source: soc_inh_neurons, var: 'g', type: parameter
    flags: set(), per_neuron: True, size: 4

  r_0_soc_exc_neurons
    source: soc_exc_neurons, var: 'r_0', type: parameter
    flags: {'shared', 'constant'}, per_neuron: False, size: 6

  r_0_soc_inh_neurons
    source: soc_inh_neurons, var: 'r_0', type: parameter
    flags: {'shared', 'constant'}, per_neuron: False, size: 4

  r_max_soc_exc_neurons
    source: soc_exc_neurons, var: 'r_max', type: parameter
    flags: {'shared', 'constant'}, per_neuron: False, size: 6

  r_max_soc_inh_neurons
    source: soc_inh_neurons, var: 'r_max', type: parameter
    flags: {'shared', 'constant'}, per_neuron: False, size: 4

  tau_soc_exc_neurons
    source: soc_exc_neurons, var: 'tau', type: parameter
    flags: {'shared'}, per_neuron: False, size: 6

  we_EE_Synapses
    source: EE_Synaps

## 9. Flat vs. Structured — Comparison

In [13]:
print("FLAT: monolithic expression with unresolved dead symbols")
print(f"  Symbols: {sorted([str(s) for s in ode_flat_exc.free_symbols])}")

print("\nSTRUCTURED: clean separation")
s = structured_exc
print(f"  Local dynamics: {s['local_expr']}")
print(f"  Coupling: {[ct['weight_symbol'].name + ' @ r(' + ct['source_group'].name + ')' for ct in s['coupling_terms']]}")
print(f"  Dead: {[str(d) for d in s['dead_symbols']]}")
print(f"  Params with metadata: {list(s['param_metadata'].keys())}")

FLAT: monolithic expression with unresolved dead symbols
  Symbols: ['I_EI_soc_exc_neurons', 'I_II_soc_exc_neurons', 'g_soc_exc_neurons', 'g_soc_inh_neurons', 'r_0_soc_exc_neurons', 'r_0_soc_inh_neurons', 'r_max_soc_exc_neurons', 'r_max_soc_inh_neurons', 'tau_soc_exc_neurons', 'we_EE_Synapses', 'wi_IE_Synapses', 'x_soc_exc_neurons', 'x_soc_inh_neurons']

STRUCTURED: clean separation
  Local dynamics: -x_soc_exc_neurons/tau_soc_exc_neurons
  Coupling: ['wi_IE_Synapses @ r(soc_inh_neurons)', 'we_EE_Synapses @ r(soc_exc_neurons)']
  Dead: ['I_II_soc_exc_neurons', 'I_EI_soc_exc_neurons']
  Params with metadata: ['g_soc_exc_neurons', 'g_soc_inh_neurons', 'r_0_soc_exc_neurons', 'r_0_soc_inh_neurons', 'r_max_soc_exc_neurons', 'r_max_soc_inh_neurons', 'tau_soc_exc_neurons', 'we_EE_Synapses', 'wi_IE_Synapses']


## 10. `discretise()` — Continuous to Discrete

| Method | Formula |
|--------|-------|
| Euler | $x_{n+1} = x_n + \Delta t \cdot F(x_n)$ |
| RK2 | $k_1 = F(x_n)$, $k_2 = F(x_n + \frac{\Delta t}{2} k_1)$, $x_{n+1} = x_n + \Delta t \cdot k_2$ |
| Exp. Euler | $x_{n+1} = x_n e^{-A\Delta t} + \frac{B}{A}(1 - e^{-A\Delta t})$ |

In [ ]:
dt_sym = sp.Symbol('dt')
x_exc = sp.Symbol(f'x_{exc.name}')

for method in ['euler', 'rk2', 'exponential_euler']:
    result = extractor.discretise(ode_flat_exc, x_exc, method, dt_sym)
    print(f"\n--- {method.upper()} ---")
    print(f"x_new = ")
    sp.pprint(sp.simplify(result), num_columns=120)
    print(f"Op count: {sp.count_ops(result)}")


--- EULER ---
x_new = 
⎧                                                                                                    dt⋅(I_EI_soc_exc_ ↪
⎪                                                                                                    ───────────────── ↪
⎪                                                                                                                      ↪
⎪                                                                                                                      ↪
⎪                                                     ⎛                                                                ↪
⎪                                                  dt⋅⎜I_EI_soc_exc_neurons + I_II_soc_exc_neurons + r_0_soc_exc_neuro ↪
⎪                                                     ⎝                                                                ↪
⎪                                                  ─────────────────────────────────────────────────────────────────── ↪
⎪       

## 11. `compute_full_jacobian()` — System-Level Derivatives

$J_{ij} = \frac{\partial F_i}{\partial x_j}$

In [ ]:
ode_flat_inh = get_derived_ode(extractor, inh, 'x')

# Zero out dead symbols for correct Jacobian
dead_subs = {
    sp.Symbol(f'I_EI_{exc.name}'): 0,
    sp.Symbol(f'I_II_{exc.name}'): 0,
    sp.Symbol(f'I_EE_{inh.name}'): 0,
    sp.Symbol(f'I_IE_{inh.name}'): 0,
}
ode_exc_clean = ode_flat_exc.subs(dead_subs)
ode_inh_clean = ode_flat_inh.subs(dead_subs)

x_e = sp.Symbol(f'x_{exc.name}')
x_i = sp.Symbol(f'x_{inh.name}')

J = compute_full_jacobian([ode_exc_clean, ode_inh_clean], [x_e, x_i])
print(f"Jacobian shape: {J.shape}")
print(f"  J[0,0] = d(F_exc)/d(x_exc): self-coupling")
print(f"  J[0,1] = d(F_exc)/d(x_inh): inh -> exc")
print(f"  J[1,0] = d(F_inh)/d(x_exc): exc -> inh")
print(f"  J[1,1] = d(F_inh)/d(x_inh): self-coupling")
for i in range(2):
    for j in range(2):
        print(f"\nJ[{i},{j}] =")
        sp.pprint(J[i, j])

## 12. Verify Symmetry — Inhibitory Group

The inh group should have `I_EI`+`I_II` as coupling and `I_EE`+`I_IE` as dead.

In [ ]:
structured_inh = extractor.get_structured_ode(inh, 'x')

print("Exc dead:", [str(d) for d in structured_exc['dead_symbols']])
print("Inh dead:", [str(d) for d in structured_inh['dead_symbols']])
print("Exc coupling:", [ct['synapse'].name for ct in structured_exc['coupling_terms']])
print("Inh coupling:", [ct['synapse'].name for ct in structured_inh['coupling_terms']])

## Summary — Data Flow

```
Brian2 Network
    |
    +-- NeuronGroups (equations, parameters)
    |       |
    |       v
    |   resolve_variable()
    |       |
    |       +-- Summed? -> _find_targeting_synapses() -> trace synapse
    |       +-- Equation? -> Diff Eq=Symbol, Subexpr=Recurse, Param=Symbol
    |       +-- Pre/Post? -> jump to source/target group
    |
    +-- get_derived_ode()      -> flat SymPy expression
    +-- get_structured_ode()   -> {local_expr, coupling_terms, dead_symbols, param_metadata}
    +-- compute_full_jacobian() -> symbolic Jacobian matrix
```